# Tracks GmbH Recruitment Test: Building a Chat Bot
### Batuhan Ipekci
*27.02.2020*


This notebook demonstrates my solution to the recruitment exam of Tracks GmbH for a Data Scientist position. It is aimed to deliver a clear explanation of the steps that are taken during the process; followed by a short discussion of what could be done in order to improve the effectiveness of the system. 

The chatbot can be started by running the script chatbot_trucks.py on the terminal. Test files for cases discussed in the seventh section are also given in the same folder. The output will always be written in the file records.txt, in the same name, replacing the results of a preceding test.

The outline is roughly given by:

1. [Description of the Task](#first)
2. [Data Scraping](#second)
3. [Text Preprocessing](#third)
4. [Text Similarity](#fourth)
5. [Misspellings and Missing Information](#fifth)
6. [Chatbot](#sixth)
7. [Tests](#seventh)
8. [Further Discussions on the Use of the Data](#eighth)






<a id='first'></a>

# 1. Description of the Task

The task is as follows:

We need to build a small interactive system (Chat Bot) to identify trucks, their specification and number in particular fleet. 
    
* Customer is Fleet Owner or Fleet Manager.
* End result of conversation should be a list of trucks with their specifications and numbers.
* All conversations should be recorded for future analysis.




My approach consists of building a data frame of truck manufacture and model names to correct misspellings, or prevent irrelevant entries. This step is very important, because we would like to gather data from the fleet managers in the cleanest way possible. The data frame has taken the function of a 'corpus' that directs the decision making.

The data is scraped, cleaned, and tokenized in the simplest possible way, given the expectations from the task. The text is normalized without using lemmatizer of stemmer, becuase there was no grammer structure to exploit as the data set contains only the names of trucks. A rule-based similarity measure (derived from Levenshtein distance) is calculated to compare user inputs with the available data frame of trucks. There is no statistical or machine learning, the system works only by heuristics. The output is attempted to be returned as human-readable as possible.

The chat bot ChatForTrucks() is constructed by considering various scenarios such as (1) if the user inputs matches with a truck model that is in our database; (2) if the user input does not match exactly, but the name of the manufacturer could be extracted, although the model name might not exist in the database; (3) if the user misspells the name of the manufacturer; (4) if the user throws something irrelevant to trucks; either by randomly writing letters, or enters a character when he/she is expected to write an integer. In addition to infering which truck the fleet manager is writing about, the knowledge on the fleet number and the specification (the total axle size) is gathered. After commanding in terminal to exit the script, the information that are gathered on the trucks are written in a file in the 'txt' format.

The cases (1), (2), (3) are dealt with figuring out the most similar words to the input in a selective way. The case (4) is solved by skipping the erroneous entries which is done simply by breaking the process if the maximal similarity to a user input is attained by more than one candidates. It is the place when human assistance is needed, and the fleet manager is directed to an actual human.

<a id='second'></a>
# 2. Data Scraping

An important task of a data scientist is to gather and clean data whenever needed. Therefore, the first table in the following Wikipedia article is scraped: https://en.wikipedia.org/wiki/List_of_trucks .

The second table is ignored in order not to spend so much time on cleaning.

As a result, a file is written which is read and manipulated in the next section.

In [205]:
import re
import urllib.request
from bs4 import BeautifulSoup

# Access to the Wikipedia Article on Truck Names
url = "https://en.wikipedia.org/wiki/List_of_trucks"
req = urllib.request.urlopen(url)
article = req.read().decode()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# For simplicity, take only the first table
ths = tables[0].find_all('th')
headline = [th.text.strip() for th in ths]

# Extract column names from one of the tables available
column_names = [re.sub(r'\s','_',headline[i].lower()) for i in [0,1]]

# Write rows (corresponding to the chosen columns) to a file
with open('truck_names.txt', 'w') as fo:
    print(';'.join(column_names), file=fo)
    for table in tables:  
        for tr in table.find_all('tr'):
            tds = tr.find_all('td')
            if not tds:
                continue
            print(';'.join(
                [ td.text.strip() for td in tds][:2]), file=fo)

<a id='third'></a>
# 3. Text Preprocessing

The usual preprocessing step for text is done through splitting sentences into tokens (it could be words, characters, or even word groups - as the n-gram approach suggests). The python nltk is a popular package to do that. Here it is avoided to overcomplicate issues by using that package, as the only aim is to compare texts of truck names.

The previously built data set is read and the relevant columns are tokenized. Those tokens are also normalized in different ways to enrich the options to match a given user input.

In [1]:
import pandas as pd
import os
import re
truck_df = pd.read_csv('truck_names.txt',sep=";")

In [2]:
truck_df.model = truck_df.model.str.replace('-',' ')

In [3]:
def tokenize_basic(text):
    '''
    This function makes user input case-insensitive.
    Additionally, it tokenizes the text 
    so that future operations can be applied easier.
    '''
    
    text = str(text)
    tokens = []
    text = re.sub(r'-|,|\.|\||_',' ',text)
    for word in text.split():
        if word.isalpha:
            word_low = word.lower()
            tokens.append(word_low)
        else:
            tokens.append(word)
    return(tokens)   

In [4]:
truck_df['model_tokens'] = truck_df.model.apply(tokenize_basic)
truck_df['model_tokens_joined'] = truck_df.model_tokens.apply(lambda x: ''.join(x))
truck_df['manufacturer_tokens'] = truck_df.manufacturer.apply(tokenize_basic)
truck_df['manufacturer_tokens_joined'] = truck_df.manufacturer_tokens.apply(lambda x: ''.join(x))
truck_df['model_number'] = truck_df.model_tokens.apply(
    lambda x:[element for element in x if element.isdigit()])
truck_df['model_number'] = truck_df.model_number.apply(
    lambda x: ''.join(x) if len(x) > 0 else '-')

In [5]:
truck_df

,manufacturer,model,model_tokens,model_tokens_joined,manufacturer_tokens,manufacturer_tokens_joined,model_number
0,American LaFrance,American Lafrance Condor,"[american, lafrance, condor]",americanlafrancecondor,"[american, lafrance]",americanlafrance,-
1,Agrale,Agrale 6500,"[agrale, 6500]",agrale6500,[agrale],agrale,6500
2,Agrale,Agrale 8700,"[agrale, 8700]",agrale8700,[agrale],agrale,8700
3,Agrale,Agrale 10000,"[agrale, 10000]",agrale10000,[agrale],agrale,10000
4,Agrale,Agrale 14000,"[agrale, 14000]",agrale14000,[agrale],agrale,14000
...,...,...,...,...,...,...,...
564,ZiL,ZIL 131,"[zil, 131]",zil131,[zil],zil,131
565,ZiL,ZiL 133,"[zil, 133]",zil133,[zil],zil,133
566,ZiL,ZiL 4334,"[zil, 4334]",zil4334,[zil],zil,4334
567,ZiL,ZiL 6309,"[zil, 6309]",zil6309,[zil],zil,6309


<a id='fourth'></a>

# 4. Text Similarity

A similarity measure is the inverse of the scaled Levenshtein distance. It is only one of the available measures and a very old one. Nevertheless, it does the function of correcting misspellings in an effective way.

Levenshtein distance is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other (See, https://en.wikipedia.org/wiki/Levenshtein_distance). In this application, it is scaled by the maximum length of compared words, so that it always remains between 0 and 1 and is comparable.

More advanced similarity measures are obtained by taking the cosine distance between word embeddings trained by some neural network, or the cosine distance between TF-IDF vectors (mostly applies to sentences). If a more advanced understanding of the text was required, those techniques would give more realistic interpretations. 

In [7]:
import textdistance

def similarity_score(s1,s2):
    '''
    This function returns a measure of a text similarity 
    based on Levenshtein distance.
    
    It is scaled by the maximum length of the compared strings. 
    The value returned is between 0 and 1, 
    therefore text similarities can be compared with each other.
    
    1 minus the scaled distance gives the similarity, just to make things intuitive.
    '''
    return(1-textdistance.levenshtein.distance(s1,s2)/max(len(s1),len(s2)))

In [28]:
similarity_score('Mercedos-Benz','Mercedes-Benz')

0.9230769230769231

<a id='fifth'></a>


# 5. Dealing with Misspellings and Missing Information


The function is written to exploit the scaled Levenshtein distance introduced in the previous section in order to give recommendations to our chat bot.

If the normalized user input directly corresponds to the normalized joint truck manufacturer-model name information, it returns the respective rows.

If not, a data frame containing the model names from the manufacturers with the most similar names to the user input.
More irrelevant an input is to our database, a bigger data frame is returned which more likely contain more than one unique manufacturers. So in this kind of situations, the chat bot stops the process and redirects the issue to a actual human. This behavior is coded in the next section.

In [9]:
def truck_recommender(tokens):
    '''
    This function takes the tokenized user as input 
    and returns a data frame of most similar 
    truck manufacturer and model name pairs.
    
    The similarity is measured by the function similarity_score,
    which is defined previously.
    '''
    user_truck = ''.join(tokens)
    
    if user_truck in truck_df.model_tokens_joined.tolist():
    # If the tokenized and joined user response happens to be in the data set,
    # return the relevant subset of the data set.
        return(truck_df[truck_df.model_tokens_joined == user_truck])
    
    else:
    # If not, collect and join all the words in the user response.
    # Then search for the most similar manufacturer (or manufacturers) 
    # to the obtained string.
    
        suggested_truck = ''.join([t for t in user_truck if t.isalpha()])
        
        # The function similarity_score is applied here:
        similarities = truck_df.manufacturer_tokens_joined.apply(
            lambda x: similarity_score(x,suggested_truck))
        
        # Collect the indices of the most similar manufacturers in the data frame
        sim_idx = similarities[similarities == similarities.max()].index.tolist()
        
        return(truck_df.iloc[sim_idx])
  

<a id='sixth'></a>


# 6. Chatbot

In [11]:
def ChatForTrucks():
    exit_flag=True
    welcome_message = """

    Welcome to the Tracks GmbH Truck Identification System

    This is your personal assistant
    In case you want to exit and save the progress, type exit

    """

    goodbye_message = "Good Bye! Drive safe and efficient..."
    truck_name_query = "Please type the manufacturer and the model of your truck. Example: Agrale 8700"
    truck_number_query = "What is the fleet number of "
    truck_spec_query = "What is the total axle load of "
    non_digit_warning = "This record will be skipped if you don\'t enter a digit"

    print(welcome_message)
    records = []
    while(exit_flag==True):
        if len(records)>0:
        # If there are already recorded trucks, print 'Next Truck' at the beginning.
            print(os.linesep + os.linesep + 'Next Truck: ' + truck_name_query)
        else:
        # Else begin with asking the truck's manufacturer its model name. 
            print(truck_name_query)

        # Tokenize the user response for the truck manufacturer and model.
        user_response = input()
        user_response_tokens = tokenize_basic(user_response)

        if 'exit' not in user_response_tokens:

            # Get a name recommendation based on Levenshtein distance
            # in case the user input cannot be matched with any manufacturer-model pairs.
            df_recommend = truck_recommender(user_response_tokens)

        ### Case 1: 1-to-1 truck manufacturer and name match
            if ''.join(user_response_tokens) in truck_df.model_tokens_joined.tolist():    
                print('The manufacturer is identified as ' + df_recommend.manufacturer.iloc[0])
                print(truck_number_query + df_recommend.model.iloc[0] + "?")
                print(non_digit_warning)

                # Proceed on requesting the fleet number in Case 1.
                fleet_number = input()
                if str(fleet_number).lower() == 'exit':
                    # Premature exit for the fleet number in Case 1.
                    # Exiting: Write the log file and give the user a notice about it.
                    print(goodbye_message + os.linesep)
                    print('... Logging ...' + os.linesep)
                    with open('records.txt', 'w') as fo:
                        print(';'.join(['manufacturer_model','fleet_number','total_axle_load']), file=fo)
                        for rec in records:
                            print(';'.join(rec), file=fo)
                    print(';'.join(['manufacturer_model','fleet_number','total_axle_load']))
                    for rec in records:
                        print(';'.join(rec))
                    exit_flag = False    

                elif fleet_number.isdigit() == False :
                    # Invalid response for the fleet number in Case 1.
                    continue
                    print(os.linesep)
                else:
                    # Valid response for the fleet number in Case 1.
                    print(truck_spec_query + df_recommend.model.iloc[0] + ' Fleet #'+ str(fleet_number))
                    print(non_digit_warning)

                    # Proceed on requesting the truck specification in Case 1.
                    truck_spec = input()
                    if str(truck_spec).lower() == 'exit':
                        # Premature exit for the truck specification in Case 1.
                        # Exiting: Write the log file and give the user a notice about it.
                        print(goodbye_message + os.linesep)
                        print('... Logging ...' + os.linesep)
                        with open('records.txt', 'w') as fo:
                            print(';'.join(['manufacturer_model','fleet_number','total_axle_load']), file=fo)
                            for rec in records:
                                print(';'.join(rec), file=fo)
                        print(';'.join(['manufacturer_model','fleet_number','total_axle_load']))
                        for rec in records:
                            print(';'.join(rec))
                        exit_flag = False

                    elif truck_spec.isdigit() == False:
                        # Invalid response for the truck specification in Case 1.
                        continue
                        print(os.linesep)
                    else:
                        # Valid response for the truck specification in Case 1.
                        print('Entry is recorded with success.')
                        print(df_recommend.model.iloc[0] +' Fleet #'+ str(fleet_number)
                              + ' Total Axle Load: ' + str(truck_spec))

                        record = [df_recommend.model.iloc[0],fleet_number,truck_spec ]
                        records.append(record)

        ### Case 2: no 1-to-1 truck manufacturer and name match
            else:
                if df_recommend.manufacturer_tokens_joined.unique().shape[0] == 1:
                # Find the most similar manufacturer (only manufacturer) to the user input in Case 2.
                    print('The manufacturer is identified as ' + df_recommend.manufacturer.iloc[0])

                    if len([t for t in user_response_tokens if t.isdigit()])==0:
                    # If the entry does not contain any number besides the manufacturer name,
                    # ask for the model number.

                        print('However, the model is missing.')
                        print('Please enter the model for '+ df_recommend.manufacturer.iloc[0] +':')
                        user_response_truck_model = input()
                        truck_model = user_response_truck_model.upper()
                        new_truck_name = df_recommend.manufacturer.iloc[0] + ' ' + truck_model
                    else:
                    # If the entry contains any number besides the manufacturer name,
                    # try to extract it without asking.

                        print('The model number you entered is new.')
                        print('It will be added to the database.')

                        remaining = re.sub(df_recommend.manufacturer_tokens_joined.iloc[0],
                                           '',''.join(user_response_tokens))
                        truck_model = remaining.upper()
                        new_truck_name = df_recommend.manufacturer.iloc[0] + ' ' + truck_model

                    print(truck_number_query + new_truck_name + ' ? ')
                    print(non_digit_warning)

                    # Proceed on requesting the fleet number for Case 2
                    fleet_number = input()
                    if str(fleet_number).lower() == 'exit':
                        # Premature exit for the fleet number in Case 2.
                        # Exiting: Write the log file and give the user a notice about it.
                        print(goodbye_message + os.linesep)
                        print('... Logging ...' + os.linesep)
                        with open('records.txt', 'w') as fo:
                            print(';'.join(['manufacturer_model','fleet_number','total_axle_load']), file=fo)
                            for rec in records:
                                print(';'.join(rec), file=fo)
                        print(';'.join(['manufacturer_model','fleet_number','total_axle_load']))
                        for rec in records:
                            print(';'.join(rec))
                        exit_flag = False   
                    elif fleet_number.isdigit() == False:
                        # Invalid response for the fleet number in Case 2.
                        continue
                        print(os.linesep)
                    else:
                        # Valid response for the fleet number in Case 2.
                        print(truck_spec_query  + new_truck_name + ' Fleet #'+ str(fleet_number) + '?')
                        print(non_digit_warning)

                        # Proceed on requesting the truck specification in Case 2.
                        truck_spec = input()
                        if str(truck_spec).lower() == 'exit':
                            # Premature exit for the truck specification in Case 2.
                            # Exiting: Write the log file and give the user a notice about it.
                            print(goodbye_message + os.linesep)
                            print('... Logging ...' + os.linesep)
                            with open('records.txt', 'w') as fo:
                                print(';'.join(['manufacturer_model','fleet_number','total_axle_load']), file=fo)
                                for rec in records:
                                    print(';'.join(rec), file=fo)
                            print(';'.join(['manufacturer_model','fleet_number','total_axle_load']))
                            for rec in records:
                                print(';'.join(rec))
                            exit_flag = False


                        elif truck_spec.isdigit() == False :
                            # Invalid response for the truck specification in Case 2.
                            continue
                            print(os.linesep)
                        else:
                            # Valid response for the truck specification in Case 2.
                            print('Entry is recorded with success.')
                            print(new_truck_name +' Fleet #'+ str(fleet_number)
                              + ' Total Axle Load: ' + str(truck_spec))

                            record = [new_truck_name ,fleet_number,truck_spec ]
                            records.append(record)

                else:
                     print('''

                     The manufacturer is not known at all.

                     Please contact to us.

                     ''')
        else:
            # Exiting: Write the log file and give the user a notice about it.
            print(goodbye_message + os.linesep)
            print('... Logging ...' + os.linesep)
            with open('records.txt', 'w') as fo:
                print(';'.join(['manufacturer_model','fleet_number','total_axle_load']), file=fo)
                for rec in records:
                    print(';'.join(rec), file=fo)
            print(';'.join(['manufacturer_model','fleet_number','total_axle_load']))
            for rec in records:
                print(';'.join(rec))
            exit_flag = False

<a id='seventh'></a>

# 7. Test Cases
For each case, print 3 trucks

1. Everything as expected

2. Manufacturer's name is in the database, but the model name is not.

3. Everything is correct except the spelling errors

4. Check the behavior when something random is given (or the case of no unique match with manufacturers).

5. Invalid response when an integer is expected

6. Exiting whenever we want

### Test 1

Everything as expected



**Input:**

    AMW TR Series
    12
    10000
    BeiBen NG80 Series
    1
    50000 
    FAW CA4252P21K2T1A
    6
    100000
    exit


**Output:**

    manufacturer_model;fleet_number;total_axle_load
    AMW TR Series;12;10000
    BeiBen NG80 Series;1;50000
    FAW CA4252P21K2T1A;6;100000

In [13]:
ChatForTrucks()



    Welcome to the Tracks GmbH Truck Identification System

    This is your personal assistant
    In case you want to exit and save the progress, type exit

    
Please type the manufacturer and the model of your truck. Example: Agrale 8700
AMW TR Series 
The manufacturer is identified as AMW
What is the fleet number of AMW TR Series?
This record will be skipped if you don't enter a digit
12
What is the total axle load of AMW TR Series Fleet #12
This record will be skipped if you don't enter a digit
10000
Entry is recorded with success.
AMW TR Series Fleet #12 Total Axle Load: 10000


Next Truck: Please type the manufacturer and the model of your truck. Example: Agrale 8700
BeiBen NG80 Series
The manufacturer is identified as BeiBen
What is the fleet number of BeiBen NG80 Series?
This record will be skipped if you don't enter a digit
1
What is the total axle load of BeiBen NG80 Series Fleet #1
This record will be skipped if you don't enter a digit
50000
Entry is recorded with succe

### Test 2

Manufacturer's name is in the database, but the model name is not.

**Input:**

    Dongfeng Dog Fighter
    Gold Fighter
    55
    7500
    Hyundai hd45345435
    hd45345435
    1
    570000
    Kenworth T4022sdsads9SAR
    T4022sdsads9SAR
    6
    121
    
**Output:**

    manufacturer_model;fleet_number;total_axle_load
    Dongfeng Gold Fighter;55;7500
    Hyundai HD45345435;1;570000
    Kenworth T4022SDSADS9SAR;6;12100

In [19]:
ChatForTrucks()



    Welcome to the Tracks GmbH Truck Identification System

    This is your personal assistant
    In case you want to exit and save the progress, type exit

    
Please type the manufacturer and the model of your truck. Example: Agrale 8700
Dongfeng Dog Fighter
The manufacturer is identified as Dongfeng
However, the model is missing.
Please enter the model for Dongfeng:
Gold Fighter
What is the fleet number of Dongfeng GOLD FIGHTER ? 
This record will be skipped if you don't enter a digit
55
What is the total axle load of Dongfeng GOLD FIGHTER Fleet #55?
This record will be skipped if you don't enter a digit
7500
Entry is recorded with success.
Dongfeng GOLD FIGHTER Fleet #55 Total Axle Load: 7500


Next Truck: Please type the manufacturer and the model of your truck. Example: Agrale 8700
Hyundai hd45345435
The manufacturer is identified as Hyundai
However, the model is missing.
Please enter the model for Hyundai:
hd45345435
What is the fleet number of Hyundai HD45345435 ? 
This re

### Test 3


Everything is correct except the spelling errors.

**Input:**

    hyund
    H350
    6
    8900
    Kenwoerurth
    T880
    9
    98000
    meister
    Forland
    34
    20000
    exit
    
**Output:**

    manufacturer_model;fleet_number;total_axle_load
    Hyundai H350;6;8900
    Kenworth T880;9;98000
    Master FORLAND;34;20000

In [20]:
ChatForTrucks()



    Welcome to the Tracks GmbH Truck Identification System

    This is your personal assistant
    In case you want to exit and save the progress, type exit

    
Please type the manufacturer and the model of your truck. Example: Agrale 8700
hyund
The manufacturer is identified as Hyundai
However, the model is missing.
Please enter the model for Hyundai:
H350
What is the fleet number of Hyundai H350 ? 
This record will be skipped if you don't enter a digit
6
What is the total axle load of Hyundai H350 Fleet #6?
This record will be skipped if you don't enter a digit
8900
Entry is recorded with success.
Hyundai H350 Fleet #6 Total Axle Load: 8900


Next Truck: Please type the manufacturer and the model of your truck. Example: Agrale 8700
Kenwoerurth
The manufacturer is identified as Kenworth
However, the model is missing.
Please enter the model for Kenworth:
T880
What is the fleet number of Kenworth T880 ? 
This record will be skipped if you don't enter a digit
9
What is the total axl

### Test 4

Something random is given (or the case of no unique match with manufacturers)


**Input:**

    nomeaninglessnesstess
    yyyyyfdnlsknla
    pokpj
    
**Output:**

    manufacturer_model;fleet_number;total_axle_load
    

In [22]:
ChatForTrucks()



    Welcome to the Tracks GmbH Truck Identification System

    This is your personal assistant
    In case you want to exit and save the progress, type exit

    
Please type the manufacturer and the model of your truck. Example: Agrale 8700
nomeaninglessnesstess


                     The manufacturer is not known at all.

                     Please contact to us.

                     
Please type the manufacturer and the model of your truck. Example: Agrale 8700
yyyyyfdnlsknla


                     The manufacturer is not known at all.

                     Please contact to us.

                     
Please type the manufacturer and the model of your truck. Example: Agrale 8700
pokpj


                     The manufacturer is not known at all.

                     Please contact to us.

                     
Please type the manufacturer and the model of your truck. Example: Agrale 8700
exit
Good Bye! Drive safe and efficient...

... Logging ...

manufacturer_model;fleet_numbe

### Test 5

Invalid response when an integer is expected



**Input:**

    MZKT - 7402
    I do not want to disclose
    International 9900i
    I said international
    Peterbilt 384
    peterbilt
    exit
    
**Output:**

    manufacturer_model;fleet_number;total_axle_load

    
    

In [23]:
ChatForTrucks()



    Welcome to the Tracks GmbH Truck Identification System

    This is your personal assistant
    In case you want to exit and save the progress, type exit

    
Please type the manufacturer and the model of your truck. Example: Agrale 8700
MZKT - 7402
The manufacturer is identified as MZKT
What is the fleet number of MZKT   7402?
This record will be skipped if you don't enter a digit
I do not want to disclose
Please type the manufacturer and the model of your truck. Example: Agrale 8700
International 9900i
The manufacturer is identified as Navistar International
What is the fleet number of International 9900i?
This record will be skipped if you don't enter a digit
I said international
Please type the manufacturer and the model of your truck. Example: Agrale 8700
Peterbilt 384
The manufacturer is identified as Peterbilt
What is the fleet number of Peterbilt 384?
This record will be skipped if you don't enter a digit
peterbilt
Please type the manufacturer and the model of your truck

### Test 6

Exiting whenever we want.

**Input:**

    MAN
    tgm
    exit
 
    
**Output:**

    manufacturer_model;fleet_number;total_axle_load

    

In [24]:
ChatForTrucks()



    Welcome to the Tracks GmbH Truck Identification System

    This is your personal assistant
    In case you want to exit and save the progress, type exit

    
Please type the manufacturer and the model of your truck. Example: Agrale 8700
MAN
The manufacturer is identified as MAN
However, the model is missing.
Please enter the model for MAN:
TGM
What is the fleet number of MAN TGM ? 
This record will be skipped if you don't enter a digit
exit
Good Bye! Drive safe and efficient...

... Logging ...

manufacturer_model;fleet_number;total_axle_load


<a id='eighth'></a>

# 8. Further Discussions on the Use of the Data

The collected data can be utilized by:

1. Merging with existing geospatial data to make interactions between trucks from the same fleet, or different fleets, possible.

2. Measuring the overall expected Green Index (?) and giving it to the fleet manager.

3. Doing ad-hoc analyses of truck specifications given the fleet.


A data collection strategy can be built by perhaps not merely asking direct questions, but considering the whole data collection process as part of the service that is given to truck managers. For instance, the truck number can be asked in response to giving a fuel consumption prediction.

# End of the Project